In [1]:
#https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt


pd.options.display.max_seq_items = 2000
pd.options.display.max_colwidth = 90
pd.options.display.max_rows = 999

#https://altair-viz.github.io/gallery/errorbars_with_std.html
import altair as alt
from vega_datasets import data

issues = './merged-closed-issues.csv'
dataIssues = pd.read_csv(issues,sep='\a')

pull_request = './merged-closed-pulls.csv'
dataPR = pd.read_csv(pull_request,sep='\a')

commits = './merged-closed-commits.csv'
dataCommit = pd.read_csv(commits,sep='\a')


#converting Float to Int 
dataIssues['Issue Number'] = dataIssues['Issue Number'].fillna(0).astype(np.int64)

#renaming collumns and excluding unecessary data
dataCommit.columns = ['PR_Number', 'Commit_Author', 'Commit_Date', 'Commit_Message', 'Commit_Patch']
dataPR.columns = ['PR_Number', 'PR_Closed_Date', 'PR_Author', 'PR_Title', 'PR_Body', 'PR_Comments', 'PR_Issue']
del dataPR['PR_Issue']
del dataCommit['Commit_Patch']
dataIssues.columns = ['PR_Number', 'Issue_Closed_Date', 'Issue_Author', 'Issue_Title', 'Issue_Body', 'Issue_Comments']

#merging dataframes
pull_commit = pd.merge(dataPR, dataCommit, on=['PR_Number','PR_Number'])



FileNotFoundError: [Errno 2] File b'./merged-closed-issues.csv' does not exist: b'./merged-closed-issues.csv'

In [ ]:
pull_commit.to_csv('pull_commit.csv', sep='\a', encoding='utf-8', index=False)

dataIssues.to_csv('issues.csv', sep='\a', encoding='utf-8', index=False)


In [ ]:
data_Issues = './issues.csv'
dataIssues = pd.read_csv(data_Issues, sep='\a' )

data_PullCommit = './pull_commit.csv'
dataPullCommit = pd.read_csv(data_PullCommit, sep='\a' )


In [ ]:
#result = pd.merge(user_usage, user_device[['use_id', 'platform', 'device']], on='use_id', how='outer',indicator=True)
result = pd.merge(dataIssues,
                 dataPullCommit,
                 on='PR_Number', 
                 how='outer', 
                 indicator=True)

In [ ]:
result.rename(columns={'_merge':'isPR'}, inplace=True)
result['isPR'].value_counts()


In [ ]:
result['isPR'] = result['isPR'].apply({'both': 1, 'left_only': 0, 'right_only':0}.get)



In [2]:
result.to_csv('merged_Data.csv', sep='\a', encoding='utf-8', index=False)


NameError: name 'result' is not defined

In [328]:
data = './merged_Data.csv'
data = pd.read_csv(data, sep='\a' )


In [329]:
data.loc[0:100,'PR_Comments']
data.loc[98,]

data.loc[data['PR_Number'] == 5000]

,PR_Number,Issue_Closed_Date,Issue_Author,Issue_Title,Issue_Body,Issue_Comments,PR_Closed_Date,PR_Author,PR_Title,PR_Body,PR_Comments,Commit_Author,Commit_Date,Commit_Message,isPR
4998,5000,NaN,wangch7,Right Click Option Menu Truncated,Version:\r\nJabRef 5.0-dev--snapshot--2019-05-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [331]:
#(data['Issue_Closed_Date']=='N/A').sum()

sum(pd.isnull(data['Issue_Closed_Date']))
data_final = data[data['Issue_Closed_Date'].notna()]

#data.Issue_Closed_Date
#data.drop(data.Issue_Closed_Date == 'NaN'].index, inplace=True)


In [332]:
data_final.to_csv('dataMerged_Final_only_closed.csv', sep='\a', encoding='utf-8', index=False)


In [333]:
data = './dataMerged_Final_only_closed.csv'
data = pd.read_csv(data, sep='\a' )

In [336]:
data

,PR_Number,Issue_Closed_Date,Issue_Author,Issue_Title,Issue_Body,Issue_Comments,PR_Closed_Date,PR_Author,PR_Title,PR_Body,PR_Comments,Commit_Author,Commit_Date,Commit_Message,isPR
0,1,2014-03-12T11:38:01Z,olenz,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",NaN,2014-03-12T11:38:01Z,olenz,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",NaN,Olaf Lenz,2014-03-12T09:16:45Z,Merge branch 'sorting',1
1,2,2014-03-12T18:29:22Z,simonharrer,Basic gradle integration,This adds basic gradle integration. The projec...,NaN,2014-03-12T18:29:22Z,simonharrer,Basic gradle integration,This adds basic gradle integration. The projec...,NaN,Simon Harrer,2014-03-12T17:31:47Z,Gradle build works basically. Reuses existing ...,1
2,3,2014-03-15T12:55:32Z,egonw,Some example new Dutch translations via the Gi...,NaN,NaN,2014-03-15T12:55:32Z,egonw,Some example new Dutch translations via the Gi...,NaN,NaN,Egon Willighagen,2014-03-15T09:33:10Z,Some example new Dutch translations via the Gi...,1
3,4,2014-03-17T22:20:50Z,jtornero,Spanish translation update,Three new strings translated.\n,=||= Even though the other translation files ...,2014-03-17T22:20:50Z,jtornero,Spanish translation update,Three new strings translated.\n,NaN,jtornero,2014-03-17T21:28:26Z,Spanish translation update,1
4,5,2014-03-18T05:12:53Z,was123,Update JabRef_in.properties,Indonesian translation added\n,NaN,2014-03-18T05:12:53Z,was123,Update JabRef_in.properties,Indonesian translation added\n,NaN,was123,2014-03-18T00:01:58Z,Update JabRef_in.properties\n\nIndonesian tran...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6088,6387,2020-05-03T09:41:42Z,wujastyk,Default bibliography mode: only bibtex,<!-- \r\nNote: Please use the GitHub issue tra...,=||= Hm. This is odd: I can choose biblatex a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6089,6389,2020-05-01T20:08:43Z,MootezSaaD,Fix underscore character being omitted from fi...,Fixes: #6383 \r\n\r\n![image](https://user-ima...,"=||= Perfect, thanks for the quick follow-up!",2020-05-01T20:08:43Z,MootezSaaD,Fix underscore character being omitted from fi...,Fixes: #6383 \r\n\r\n![image](https://user-ima...,=||= Could you please add a short comment exp...,Mootez,2020-05-01T17:49:28Z,Comment why mnemonic parsing is disabled,1
6090,6393,2020-05-02T11:00:56Z,LuckyOne09,Update code-howtos.md,fix two typos in code-howtos.md\r\n\r\n<!-- \r...,=||= Thanks!,2020-05-02T11:00:56Z,LuckyOne09,Update code-howtos.md,fix two typos in code-howtos.md\r\n\r\n<!-- \r...,NaN,Yunsheng Luo,2020-05-02T08:33:28Z,Update code-howtos.md\n\nfix two typos in code...,1
6091,6396,2020-05-03T18:43:33Z,HifeFish,StartupWMclass org.jabref.JabRefMain vs org-ja...,I installed JabRef 5.0 from the .deb package o...,=||= @HifeFish Thanks for reporting that iss...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [9]:
teste = './df_merge.csv'
testeMerge = pd.read_csv(teste,sep=',')
testeMerge

,PR_Number,Issue_Title,Issue_Body,Issue_Comments,PR_Comments,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
0,1,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,NaN,Merge branch sorting,1,NaN,NaN,NaN,NaN,NaN,1.0
1,2,Basic gradle integration,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,NaN,Gradle build works basically. Reuses existing folder structure.,1,NaN,NaN,NaN,NaN,NaN,1.0
2,3,Some example new Dutch translations via the GitHub web page.,NaN,NaN,NaN,Some example new Dutch translations via the GitHub web page.,1,NaN,NaN,NaN,NaN,NaN,1.0
3,4,Spanish translation update,Three new strings translated.,=||= Even though the other translation files have been updated I accept this pull req...,NaN,Spanish translation update,1,NaN,NaN,NaN,NaN,NaN,1.0
4,5,Update JabRef_in.properties,Indonesian translation added,NaN,NaN,Update JabRef_in.propertiesIndonesian translation added,1,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6489,6387,Default bibliography mode: only bibtex,<!-- Note: Please use the GitHub issue tracker only for bug reports. Feature requests ...,=||= Hm. This is odd: I can choose biblatex as well.I also did a reset of my preferen...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1.0
6490,6389,Fix underscore character being omitted from file name in Recent Libraries list,Fixes: #6383 ![image](https://user-images.githubusercontent.com/34676841/80824286-6b74...,=||= Perfect thanks for the quick follow-up!,=||= Could you please add a short comment explaining why mnemonic parsing is disabled...,Comment why mnemonic parsing is disabled,1,6389.0,6383.0,Recent Libraries list in File menu omits underscores in filenames,<!-- Note: Please use the GitHub issue tracker only for bug reports. Feature requests ...,=||= By default `mnemonicParsingProperty` is set to `true` for MenuItem which parses ...,0.0
6491,6393,Update code-howtos.md,fix two typos in code-howtos.md<!-- Describe the changes you have made here: what why ...,=||= Thanks!,NaN,Update code-howtos.mdfix two typos in code-howtos.md,1,NaN,NaN,NaN,NaN,NaN,1.0
6492,6396,StartupWMclass org.jabref.JabRefMain vs org-jabref-JabRefMain,I installed JabRef 5.0 from the .deb package on Ubuntu 16.04.JabRef is not correctly a...,=||= @HifeFish Thanks for reporting that issue. It would be nice if you could chang...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1.0


In [12]:
#testeMerge.loc[7,]

#testeMerge.loc[testeMerge['PR_Number'] == 4498]

#testeMerge.loc[testeMerge['PR_Number'] == 4188]

testeMerge.loc[testeMerge['PR_Number'] == 452]

,PR_Number,Issue_Title,Issue_Body,Issue_Comments,PR_Comments,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
452,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,315.0,Rating colum has no header description,We should add a description here especially since clicking inside the column always op...,=||= The reason for directly changing the flags convenience for quick changing of fla...,0.0
453,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,384.0,Tooltip for title or long field,Adding a tooltip so we know the meaning of each column headtitle (R R Q F P is not obv...,=||= Thanks for your input. This issue is actively discussed among the developers so ...,0.0
454,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,443.0,GUI: Clicking on Ranking Col Header switches position of DOI column before/after Entry...,In 3.0 I just discovered a strange behavior:When I click on the Ranking Column Header ...,=||= Confirmed for 3.0 and `master`. =||= Fixed with #452 - changes are available now...,0.0
455,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,445.0,Moving ranking column: moved visually but function still in old place,In JabRef 3.0 I moved the ranking column to the end of my entry table using drag and d...,=||= Possibly #443 is related. =||= Fixed with #452 - changes are available now in cu...,0.0
456,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,498.0,Open pdf file when clicking pdf icon,NaN,=||= ... is working fine.Seriously you expect us to invest our time for free on somet...,0.0
457,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,490.0,Main Table: Right click on DOI field: Allow copying DOI,Currently the popup of DOI allows for clicking the DOI only. It should also be possibl...,=||= On which field? In field editor? See #492 =||= In the main table. At the DOI col...,0.0


In [11]:
testeMerge.loc[testeMerge['PR_Number'] == 677]

,PR_Number,Issue_Title,Issue_Body,Issue_Comments,PR_Comments,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
681,677,Make BibTex parser more robust against missing newlines,As described in #669 and #621 the parser has some problems when an entry (or preamble)...,=||= :+1: (except my minor comment) =||= Corrected... for some reason the code style ...,=||= Please always use curly braces as this makes the semantics explicit.,Minor code reformatting,1,677.0,669.0,JabRef 3.1/3.2 writes bib file in a format that it will not read,If the @PREAMBLE field is present JabRef 3.1/3.2 omits the newline between that line a...,=||= related #621 =||= @tobiasdiez Is this fixed? =||= @stefan-kolb It will be fixed ...,0.0
682,677,Make BibTex parser more robust against missing newlines,As described in #669 and #621 the parser has some problems when an entry (or preamble)...,=||= :+1: (except my minor comment) =||= Corrected... for some reason the code style ...,=||= Please always use curly braces as this makes the semantics explicit.,Minor code reformatting,1,677.0,621.0,Ver 3.1 comment not written correctly.,I have a file started with bibtex item e.g.1 @ARTICLE{XXXXXX2 author = {XXXXXXXX...,=||= This is not a bug but the exact desired behavior.We only modify entries that hav...,0.0
683,677,Make BibTex parser more robust against missing newlines,As described in #669 and #621 the parser has some problems when an entry (or preamble)...,=||= :+1: (except my minor comment) =||= Corrected... for some reason the code style ...,=||= Please always use curly braces as this makes the semantics explicit.,Minor code reformatting,1,677.0,669.0,JabRef 3.1/3.2 writes bib file in a format that it will not read,If the @PREAMBLE field is present JabRef 3.1/3.2 omits the newline between that line a...,=||= related #621 =||= @tobiasdiez Is this fixed? =||= @stefan-kolb It will be fixed ...,0.0
684,677,Make BibTex parser more robust against missing newlines,As described in #669 and #621 the parser has some problems when an entry (or preamble)...,=||= :+1: (except my minor comment) =||= Corrected... for some reason the code style ...,=||= Please always use curly braces as this makes the semantics explicit.,Minor code reformatting,1,677.0,621.0,Ver 3.1 comment not written correctly.,I have a file started with bibtex item e.g.1 @ARTICLE{XXXXXX2 author = {XXXXXXXX...,=||= This is not a bug but the exact desired behavior.We only modify entries that hav...,0.0


,PR_Number,Issue_Title,Issue_Body,Issue_Comments,PR_Comments,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
